In [0]:
import pandas as pd

In [90]:
data_filename = "games0.csv"
dataset = pd.read_csv(data_filename)
dataset.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,a,b,Notes
0,Tue Oct 17 2017,8:01p,Boston Celtics,99,Cleveland Cavaliers,102,Box Score,NaN,NaN
1,Tue Oct 17 2017,10:30p,Houston Rockets,122,Golden State Warriors,121,Box Score,NaN,NaN
2,Wed Oct 18 2017,7:00p,Charlotte Hornets,90,Detroit Pistons,102,Box Score,NaN,NaN
3,Wed Oct 18 2017,7:00p,Brooklyn Nets,131,Indiana Pacers,140,Box Score,NaN,NaN
4,Wed Oct 18 2017,7:00p,Miami Heat,109,Orlando Magic,116,Box Score,NaN,NaN


In [0]:
#test = pd.read_csv(data_filename, parse_dates=["Date"])
#test.drop(columns='Attend.', inplace=True)
#test.columns = ["Date", "Start (ET)", "Visitor Team", "VisitorPts", "Home Team", "HomePts", "OT?", "Score type", "Notes"]

In [0]:
#test.head()

In [0]:
dataset = pd.read_csv(data_filename, parse_dates=["Date"])
dataset.columns = ["Date", "Start (ET)", "Visitor Team", "VisitorPts", "Home Team", "HomePts", "OT?", "Score type", "Notes"]

In [92]:
print(dataset)

          Date Start (ET)           Visitor Team  ...        OT? Score type  Notes
0   2017-10-17      8:01p         Boston Celtics  ...  Box Score        NaN    NaN
1   2017-10-17     10:30p        Houston Rockets  ...  Box Score        NaN    NaN
2   2017-10-18      7:00p      Charlotte Hornets  ...  Box Score        NaN    NaN
3   2017-10-18      7:00p          Brooklyn Nets  ...  Box Score        NaN    NaN
4   2017-10-18      7:00p             Miami Heat  ...  Box Score        NaN    NaN
..         ...        ...                    ...  ...        ...        ...    ...
99  2017-10-30     10:30p  Golden State Warriors  ...  Box Score        NaN    NaN
100 2017-10-31      7:00p       Sacramento Kings  ...  Box Score        NaN    NaN
101 2017-10-31      7:30p           Phoenix Suns  ...  Box Score        NaN    NaN
102 2017-10-31      8:00p  Oklahoma City Thunder  ...  Box Score        NaN    NaN
103 2017-10-31     10:30p        Detroit Pistons  ...  Box Score        NaN    NaN

[10

In [93]:
print(dataset.dtypes)

Date            datetime64[ns]
Start (ET)              object
Visitor Team            object
VisitorPts               int64
Home Team               object
HomePts                  int64
OT?                     object
Score type              object
Notes                  float64
dtype: object


In [0]:
dataset["HomeWin"] = dataset["VisitorPts"] < dataset["HomePts"]

In [95]:
dataset.head()

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Score type,Notes,HomeWin
0,2017-10-17,8:01p,Boston Celtics,99,Cleveland Cavaliers,102,Box Score,NaN,NaN,True
1,2017-10-17,10:30p,Houston Rockets,122,Golden State Warriors,121,Box Score,NaN,NaN,False
2,2017-10-18,7:00p,Charlotte Hornets,90,Detroit Pistons,102,Box Score,NaN,NaN,True
3,2017-10-18,7:00p,Brooklyn Nets,131,Indiana Pacers,140,Box Score,NaN,NaN,True
4,2017-10-18,7:00p,Miami Heat,109,Orlando Magic,116,Box Score,NaN,NaN,True


In [0]:
y_true = dataset["HomeWin"].values

In [97]:
dataset["HomeWin"].mean()

0.5480769230769231

In [0]:
from collections import defaultdict
won_last = defaultdict(int)

In [0]:
dataset["HomeLastWin"] = 0
dataset["VisitorLastWin"] = 0

In [100]:
for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeLastWin"] = won_last[home_team]
    #dataset.set_value(index, "HomeLastWin", won_last[home_team])
    dataset.at[index, "HomeLastWin"] = won_last[home_team]
    #dataset.set_value(index, "VisitorLastWin", won_last[visitor_team])
    dataset.at[index, "VisitorLastWin"] = won_last[visitor_team]

    won_last[home_team] = int(row["HomeWin"])
    won_last[visitor_team] = 1 - int(row["HomeWin"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [0]:
dataset.head(15)

In [0]:
X_previouswins = dataset[["HomeLastWin", "VisitorLastWin"]].values

In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=14)

In [0]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [107]:
scores = cross_val_score(clf, X_previouswins, y_true,
scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 53.0%


In [0]:
standings = pd.read_csv("st.csv", skiprows=1)
print(standings)

In [0]:
#standings.drop([21], axis=0, inplace=True)
#print(standings)

In [0]:
#standings.to_csv('test.csv')

In [0]:
#st =  pd.read_csv("test.csv")
#st.head()

In [0]:
#stn = st.rename(columns={'RK': 'RK2'})
#stn = st.rename(columns={'Unnamed: 0': 'RK'})
#print(stn)

In [0]:
#from collections import defaultdict
dataset["HomeTeamRanksHigher"] = 0

In [110]:
for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    home_rank = standings[standings["Team"] == home_team]["Rk"].values[0]
    visitor_rank = standings[standings["Team"] == visitor_team]["Rk"].values[0]
    dataset.set_value(index, "HomeTeamRanksHigher", int(home_rank < visitor_rank))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [0]:
X_homehigher = dataset[[ "HomeTeamRanksHigher", "HomeLastWin", "VisitorLastWin",]].values

In [112]:
clf = DecisionTreeClassifier(random_state=14, criterion="entropy")

scores = cross_val_score(clf, X_homehigher, y_true, scoring='accuracy')

print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 66.3%


In [113]:
last_match_winner = defaultdict(int)
dataset["HomeTeamWonLast"] = 0

for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    teams = tuple(sorted([home_team, visitor_team]))  # Sort for a consistent ordering
    # Set in the row, who won the last encounter
    home_team_won_last = 1 if last_match_winner[teams] == row["Home Team"] else 0
    #dataset.set_value(index, "HomeTeamWonLast", home_team_won_last)
    dataset.at[index, "HomeTeamWonLast"] = home_team_won_last

    # Who won this one?
    winner = row["Home Team"] if row["HomeWin"] else row["Visitor Team"]
    last_match_winner[teams] = winner

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


In [0]:
dataset.iloc[14:17]

In [0]:
#dataset.ix[14:25]
dataset.loc[14:25]

In [115]:
X_lastwinner = dataset[[ "HomeTeamWonLast", "HomeTeamRanksHigher", "HomeLastWin", "VisitorLastWin",]].values
clf = DecisionTreeClassifier(random_state=14, criterion="entropy")

scores = cross_val_score(clf, X_lastwinner, y_true, scoring='accuracy')

print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 63.4%


In [0]:
from sklearn.preprocessing import LabelEncoder
encoding = LabelEncoder()
encoding.fit(dataset["Home Team"].values)
home_teams = encoding.transform(dataset["Home Team"].values)
visitor_teams = encoding.transform(dataset["Visitor Team"].values)
X_teams = np.vstack([home_teams, visitor_teams]).T

In [118]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
X_teams = onehot.fit_transform(X_teams).todense()

clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 47.0%


In [119]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 46.0%


In [120]:
X_all = np.hstack([X_lastwinner, X_teams])
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_all, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 58.5%


In [121]:
X_all = np.hstack([X_lastwinner, X_teams])
clf = RandomForestClassifier(random_state=14, n_estimators=250)
scores = cross_val_score(clf, X_all, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 57.6%


In [122]:
from sklearn.model_selection import GridSearchCV
parameter_space = {
    "max_features": [2, 10, 'auto'],
    "n_estimators": [100, 200],
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": [2, 4, 6],
}
clf = RandomForestClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_all, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))

Accuracy: 56.6%


In [123]:
print(grid.best_estimator_)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=10,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=14, verbose=0,
                       warm_start=False)
